In [31]:
import re
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import sigmoid_kernel

In [32]:
initial_df = pd.read_csv(r"C:\Users\HP\Downloads\steam_games.csv")
initial_df

,url,types,name,desc_snippet,recent_reviews,all_reviews,release_date,developer,publisher,popular_tags,game_details,languages,achievements,genre,game_description,mature_content,minimum_requirements,recommended_requirements,original_price,discount_price
0,https://store.steampowered.com/app/379720/DOOM/,app,DOOM,Now includes all three premium DLC packs (Unto...,"Very Positive,(554),- 89% of the 554 user revi...","Very Positive,(42,550),- 92% of the 42,550 use...","May 12, 2016",id Software,"Bethesda Softworks,Bethesda Softworks","FPS,Gore,Action,Demons,Shooter,First-Person,Gr...","Single-player,Multi-player,Co-op,Steam Achieve...","English,French,Italian,German,Spanish - Spain,...",54.0,Action,"About This Game Developed by id software, the...",NaN,"Minimum:,OS:,Windows 7/8.1/10 (64-bit versions...","Recommended:,OS:,Windows 7/8.1/10 (64-bit vers...",$19.99,$14.99
1,https://store.steampowered.com/app/578080/PLAY...,app,PLAYERUNKNOWN'S BATTLEGROUNDS,PLAYERUNKNOWN'S BATTLEGROUNDS is a battle roya...,"Mixed,(6,214),- 49% of the 6,214 user reviews ...","Mixed,(836,608),- 49% of the 836,608 user revi...","Dec 21, 2017",PUBG Corporation,"PUBG Corporation,PUBG Corporation","Survival,Shooter,Multiplayer,Battle Royale,PvP...","Multi-player,Online Multi-Player,Stats","English,Korean,Simplified Chinese,French,Germa...",37.0,"Action,Adventure,Massively Multiplayer",About This Game PLAYERUNKNOWN'S BATTLEGROUND...,Mature Content Description The developers de...,"Minimum:,Requires a 64-bit processor and opera...","Recommended:,Requires a 64-bit processor and o...",$29.99,NaN
2,https://store.steampowered.com/app/637090/BATT...,app,BATTLETECH,Take command of your own mercenary outfit of '...,"Mixed,(166),- 54% of the 166 user reviews in t...","Mostly Positive,(7,030),- 71% of the 7,030 use...","Apr 24, 2018",Harebrained Schemes,"Paradox Interactive,Paradox Interactive","Mechs,Strategy,Turn-Based,Turn-Based Tactics,S...","Single-player,Multi-player,Online Multi-Player...","English,French,German,Russian",128.0,"Action,Adventure,Strategy",About This Game From original BATTLETECH/Mec...,NaN,"Minimum:,Requires a 64-bit processor and opera...","Recommended:,Requires a 64-bit processor and o...",$39.99,NaN
3,https://store.steampowered.com/app/221100/DayZ/,app,DayZ,The post-soviet country of Chernarus is struck...,"Mixed,(932),- 57% of the 932 user reviews in t...","Mixed,(167,115),- 61% of the 167,115 user revi...","Dec 13, 2018",Bohemia Interactive,"Bohemia Interactive,Bohemia Interactive","Survival,Zombies,Open World,Multiplayer,PvP,Ma...","Multi-player,Online Multi-Player,Steam Worksho...","English,French,Italian,German,Spanish - Spain,...",NaN,"Action,Adventure,Massively Multiplayer",About This Game The post-soviet country of Ch...,NaN,"Minimum:,OS:,Windows 7/8.1 64-bit,Processor:,I...","Recommended:,OS:,Windows 10 64-bit,Processor:,...",$44.99,NaN
4,https://store.steampowered.com/app/8500/EVE_On...,app,EVE Online,EVE Online is a community-driven spaceship MMO...,"Mixed,(287),- 54% of the 287 user reviews in t...","Mostly Positive,(11,481),- 74% of the 11,481 u...","May 6, 2003",CCP,"CCP,CCP","Space,Massively Multiplayer,Sci-fi,Sandbox,MMO...","Multi-player,Online Multi-Player,MMO,Co-op,Onl...","English,German,Russian,French",NaN,"Action,Free to Play,Massively Multiplayer,RPG,...",About This Game,NaN,"Minimum:,OS:,Windows 7,Processor:,Intel Dual C...","Recommended:,OS:,Windows 10,Processor:,Intel i...",Free,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40828,https://store.steampowered.com/app/899836/Rock...,app,Rocksmith® 2014 Edition – Remastered – Sabaton...,NaN,NaN,NaN,"Feb 12, 2019",Ubisoft - San Francisco,NaN,"Casual,Simulation","Single-player,Shared/Split Screen,Downloadable...","English,German,French,Italian,Spanish - Spain,...",NaN,"Casual,Simulation","About This Content Play ""Ghost Division"" by S...",NaN,"Minimum:,OS:,Windows Vista, Windows 7, Windows...","Recommended:,OS:,Windows Vista, Windows 7,

In [33]:
df_games = pd.read_csv(r"C:\Users\HP\Downloads\steam_games.csv",
                     usecols=["name", "genre", "game_details", "popular_tags", "publisher", "developer"])
df_games

,name,developer,publisher,popular_tags,game_details,genre
0,DOOM,id Software,"Bethesda Softworks,Bethesda Softworks","FPS,Gore,Action,Demons,Shooter,First-Person,Gr...","Single-player,Multi-player,Co-op,Steam Achieve...",Action
1,PLAYERUNKNOWN'S BATTLEGROUNDS,PUBG Corporation,"PUBG Corporation,PUBG Corporation","Survival,Shooter,Multiplayer,Battle Royale,PvP...","Multi-player,Online Multi-Player,Stats","Action,Adventure,Massively Multiplayer"
2,BATTLETECH,Harebrained Schemes,"Paradox Interactive,Paradox Interactive","Mechs,Strategy,Turn-Based,Turn-Based Tactics,S...","Single-player,Multi-player,Online Multi-Player...","Action,Adventure,Strategy"
3,DayZ,Bohemia Interactive,"Bohemia Interactive,Bohemia Interactive","Survival,Zombies,Open World,Multiplayer,PvP,Ma...","Multi-player,Online Multi-Player,Steam Worksho...","Action,Adventure,Massively Multiplayer"
4,EVE Online,CCP,"CCP,CCP","Space,Massively Multiplayer,Sci-fi,Sandbox,MMO...","Multi-player,Online Multi-Player,MMO,Co-op,Onl...","Action,Free to Play,Massively Multiplayer,RPG,..."
...,...,...,...,...,...,...
40828,Rocksmith® 2014 Edition – Remastered – Sabaton...,Ubisoft - San Francisco,NaN,"Casual,Simulation","Single-player,Shared/Split Screen,Downloadable...","Casual,Simulation"
40829,Rocksmith® 2014 Edition – Remastered – Stone T...,Ubisoft - San Francisco,NaN,"Casual,Simulation","Single-player,Shared/Split Screen,Downloadable...","Casual,Simulation"
40830,Fantasy Grounds - Quests of Doom 4: A Midnight...,"SmiteWorks USA, LLC",NaN,"RPG,Indie,Strategy,Software,Turn-Based,Fantasy...","Multi-player,Co-op,Cross-Platform Multiplayer,...","Indie,RPG,Strategy"
40831,Mega Man X5 Sound Collection,"CAPCOM CO., LTD","CAPCOM CO., LTD,CAPCOM CO., LTD",Action,"Single-player,Downloadable Content,Steam Achie...",Action


In [34]:
initial_users = pd.read_csv(r'C:\Users\HP\Downloads\steam_users_purchase_play.csv')
initial_users

,user_id,game_name,hours,purchase,play
0,151603712,The Elder Scrolls V Skyrim,273.0,1,1
1,151603712,Fallout 4,87.0,1,1
2,151603712,Spore,14.9,1,1
3,151603712,Fallout New Vegas,12.1,1,1
4,151603712,Left 4 Dead 2,8.9,1,1
...,...,...,...,...,...
128799,128470551,Fallen Earth,2.4,1,1
128800,128470551,Magic Duels,2.2,1,1
128801,128470551,Titan Souls,1.5,1,1
128802,128470551,Grand Theft Auto Vice City,1.5,1,1


In [35]:
df_users = pd.read_csv(r'C:\Users\HP\Downloads\steam_users_purchase_play.csv', usecols=[0, 1, 2, 3],)
df_users

,user_id,game_name,hours,purchase
0,151603712,The Elder Scrolls V Skyrim,273.0,1
1,151603712,Fallout 4,87.0,1
2,151603712,Spore,14.9,1
3,151603712,Fallout New Vegas,12.1,1
4,151603712,Left 4 Dead 2,8.9,1
...,...,...,...,...
128799,128470551,Fallen Earth,2.4,1
128800,128470551,Magic Duels,2.2,1
128801,128470551,Titan Souls,1.5,1
128802,128470551,Grand Theft Auto Vice City,1.5,1


In [36]:
df_games['name'] = df_games['name'].fillna('')
df_games["ID"] = ""
df_users["ID"] = ""

In [37]:

for i, row in df_games.iterrows():
    clean = re.sub('[^A-Za-z0-9]+', '', row["name"])
    clean = clean.lower()
    df_games.at[i, 'ID'] = clean
    
for i, row in df_users.iterrows():
    clean = re.sub('[^A-Za-z0-9]+', '', row["game_name"])
    clean = clean.lower()
    df_users.at[i, 'ID'] = clean

In [38]:

game_array_users = df_users["ID"].unique()
print(len(game_array_users))
criteria_test = df_games['ID'].isin(game_array_users)
df_used_games = df_games[criteria_test]
print(len(df_used_games))

5151
3036


In [39]:


df_used_games.loc[:, 'genre'] = df_used_games['genre'].fillna('')
df_used_games.loc[:, 'game_details'] = df_used_games['game_details'].fillna('')
df_used_games.loc[:, 'popular_tags'] = df_used_games['popular_tags'].fillna('')
df_used_games.loc[:, 'publisher'] = df_used_games['publisher'].fillna('')
df_used_games.loc[:, 'developer'] = df_used_games['developer'].fillna('')

c:\users\hp\appdata\local\programs\python\python39\lib\site-packages\pandas\core\indexing.py:1843: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value


In [40]:
def clean_data(x):
    if isinstance(x, str):
        return x.replace(" ", "")
    else:
        print(x)
        return x

In [41]:
df_used_games.loc[:, 'genre'] = df_used_games['genre'].apply(clean_data)
df_used_games.loc[:, 'game_details'] = df_used_games['game_details'].apply(clean_data)
df_used_games.loc[:, 'popular_tags'] = df_used_games['popular_tags'].apply(clean_data)
df_used_games.loc[:, 'publisher'] = df_used_games['publisher'].apply(clean_data)
df_used_games.loc[:, 'developer'] = df_used_games['developer'].apply(clean_data)


In [42]:
df_used_games["genre_publisher_developer"] = df_used_games['genre'] + df_used_games['publisher'] + df_used_games['developer']
df_used_games["genre_popular_tags_developer"] = df_used_games['genre'] + df_used_games['popular_tags'] + df_used_games['developer']
df_used_games["genre_popular_tags_game_details"] = df_used_games['genre'] + df_used_games['popular_tags'] + df_used_games['game_details']
df_used_games["genre_publisher_developer_game_details"] = df_used_games['genre'] + df_used_games['publisher'] + df_used_games['developer'] + df_used_games['game_details']


<ipython-input-42-8172b51cf225>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_used_games["genre_publisher_developer"] = df_used_games['genre'] + df_used_games['publisher'] + df_used_games['developer']
<ipython-input-42-8172b51cf225>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_used_games["genre_popular_tags_developer"] = df_used_games['genre'] + df_used_games['popular_tags'] + df_used_games['developer']
<ipython-input-42-8172b51cf225>:3: SettingWithCopyWarning: 
A value is trying to be set on a

In [43]:
df_used_games.drop_duplicates("name")
df_used_games.to_csv(r'C:\Users\HP\Downloads\processed_games_for_content_based.csv', index=False)


In [44]:
final_df=pd.read_csv(r"C:\Users\HP\Downloads\processed_games_for_content_based.csv")
final_df.head(5)

,name,developer,publisher,popular_tags,game_details,genre,ID,genre_publisher_developer,genre_popular_tags_developer,genre_popular_tags_game_details,genre_publisher_developer_game_details
0,DayZ,BohemiaInteractive,"BohemiaInteractive,BohemiaInteractive","Survival,Zombies,OpenWorld,Multiplayer,PvP,Mas...","Multi-player,OnlineMulti-Player,SteamWorkshop,...","Action,Adventure,MassivelyMultiplayer",dayz,"Action,Adventure,MassivelyMultiplayerBohemiaIn...","Action,Adventure,MassivelyMultiplayerSurvival,...","Action,Adventure,MassivelyMultiplayerSurvival,...","Action,Adventure,MassivelyMultiplayerBohemiaIn..."
1,EVE Online,CCP,"CCP,CCP","Space,MassivelyMultiplayer,Sci-fi,Sandbox,MMOR...","Multi-player,OnlineMulti-Player,MMO,Co-op,Onli...","Action,FreetoPlay,MassivelyMultiplayer,RPG,Str...",eveonline,"Action,FreetoPlay,MassivelyMultiplayer,RPG,Str...","Action,FreetoPlay,MassivelyMultiplayer,RPG,Str...","Action,FreetoPlay,MassivelyMultiplayer,RPG,Str...","Action,FreetoPlay,MassivelyMultiplayer,RPG,Str..."
2,TERA,"Bluehole,Inc.","EnMasseEntertainment,EnMasseEntertainment","FreetoPlay,MMORPG,MassivelyMultiplayer,RPG,Ope...","Multi-player,MMO,Co-op,SteamTradingCards,Parti...","Action,Adventure,FreetoPlay,MassivelyMultiplay...",tera,"Action,Adventure,FreetoPlay,MassivelyMultiplay...","Action,Adventure,FreetoPlay,MassivelyMultiplay...","Action,Adventure,FreetoPlay,MassivelyMultiplay...","Action,Adventure,FreetoPlay,MassivelyMultiplay..."
3,Stonehearth,RadiantEntertainment,"(none),(none)","CityBuilder,Building,Sandbox,Strategy,Survival...","Single-player,Multi-player,OnlineMulti-Player,...","Indie,Simulation,Strategy",stonehearth,"Indie,Simulation,Strategy(none),(none)RadiantE...","Indie,Simulation,StrategyCityBuilder,Building,...","Indie,Simulation,StrategyCityBuilder,Building,...","Indie,Simulation,Strategy(none),(none)RadiantE..."
4,Call of Duty®: Black Ops,Treyarch,"Activision,Activision","Action,FPS,Zombies,Multiplayer,Shooter,Singlep...","Single-player,Multi-player,Co-op,SteamAchievem...",Action,callofdutyblackops,"ActionActivision,ActivisionTreyarch","ActionAction,FPS,Zombies,Multiplayer,Shooter,S...","ActionAction,FPS,Zombies,Multiplayer,Shooter,S...","ActionActivision,ActivisionTreyarchSingle-play..."


In [45]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3036 entries, 0 to 3035
Data columns (total 11 columns):
 #   Column                                  Non-Null Count  Dtype 
---  ------                                  --------------  ----- 
 0   name                                    3036 non-null   object
 1   developer                               3012 non-null   object
 2   publisher                               2963 non-null   object
 3   popular_tags                            3033 non-null   object
 4   game_details                            3021 non-null   object
 5   genre                                   3018 non-null   object
 6   ID                                      3036 non-null   object
 7   genre_publisher_developer               3035 non-null   object
 8   genre_popular_tags_developer            3036 non-null   object
 9   genre_popular_tags_game_details         3036 non-null   object
 10  genre_publisher_developer_game_details  3035 non-null   object
dtypes: o

In [46]:
final_df.dropna(axis=0,inplace=True)

In [47]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2919 entries, 0 to 3035
Data columns (total 11 columns):
 #   Column                                  Non-Null Count  Dtype 
---  ------                                  --------------  ----- 
 0   name                                    2919 non-null   object
 1   developer                               2919 non-null   object
 2   publisher                               2919 non-null   object
 3   popular_tags                            2919 non-null   object
 4   game_details                            2919 non-null   object
 5   genre                                   2919 non-null   object
 6   ID                                      2919 non-null   object
 7   genre_publisher_developer               2919 non-null   object
 8   genre_popular_tags_developer            2919 non-null   object
 9   genre_popular_tags_game_details         2919 non-null   object
 10  genre_publisher_developer_game_details  2919 non-null   object
dtypes: o

In [48]:
final_df.reset_index(drop=True)

,name,developer,publisher,popular_tags,game_details,genre,ID,genre_publisher_developer,genre_popular_tags_developer,genre_popular_tags_game_details,genre_publisher_developer_game_details
0,DayZ,BohemiaInteractive,"BohemiaInteractive,BohemiaInteractive","Survival,Zombies,OpenWorld,Multiplayer,PvP,Mas...","Multi-player,OnlineMulti-Player,SteamWorkshop,...","Action,Adventure,MassivelyMultiplayer",dayz,"Action,Adventure,MassivelyMultiplayerBohemiaIn...","Action,Adventure,MassivelyMultiplayerSurvival,...","Action,Adventure,MassivelyMultiplayerSurvival,...","Action,Adventure,MassivelyMultiplayerBohemiaIn..."
1,EVE Online,CCP,"CCP,CCP","Space,MassivelyMultiplayer,Sci-fi,Sandbox,MMOR...","Multi-player,OnlineMulti-Player,MMO,Co-op,Onli...","Action,FreetoPlay,MassivelyMultiplayer,RPG,Str...",eveonline,"Action,FreetoPlay,MassivelyMultiplayer,RPG,Str...","Action,FreetoPlay,MassivelyMultiplayer,RPG,Str...","Action,FreetoPlay,MassivelyMultiplayer,RPG,Str...","Action,FreetoPlay,MassivelyMultiplayer,RPG,Str..."
2,TERA,"Bluehole,Inc.","EnMasseEntertainment,EnMasseEntertainment","FreetoPlay,MMORPG,MassivelyMultiplayer,RPG,Ope...","Multi-player,MMO,Co-op,SteamTradingCards,Parti...","Action,Adventure,FreetoPlay,MassivelyMultiplay...",tera,"Action,Adventure,FreetoPlay,MassivelyMultiplay...","Action,Adventure,FreetoPlay,MassivelyMultiplay...","Action,Adventure,FreetoPlay,MassivelyMultiplay...","Action,Adventure,FreetoPlay,MassivelyMultiplay..."
3,Stonehearth,RadiantEntertainment,"(none),(none)","CityBuilder,Building,Sandbox,Strategy,Survival...","Single-player,Multi-player,OnlineMulti-Player,...","Indie,Simulation,Strategy",stonehearth,"Indie,Simulation,Strategy(none),(none)RadiantE...","Indie,Simulation,StrategyCityBuilder,Building,...","Indie,Simulation,StrategyCityBuilder,Building,...","Indie,Simulation,Strategy(none),(none)RadiantE..."
4,Call of Duty®: Black Ops,Treyarch,"Activision,Activision","Action,FPS,Zombies,Multiplayer,Shooter,Singlep...","Single-player,Multi-player,Co-op,SteamAchievem...",Action,callofdutyblackops,"ActionActivision,ActivisionTreyarch","ActionAction,FPS,Zombies,Multiplayer,Shooter,S...","ActionAction,FPS,Zombies,Multiplayer,Shooter,S...","ActionActivision,ActivisionTreyarchSingle-play..."
...,...,...,...,...,...,...,...,...,...,...,...
2914,RIP,InvertedSouls,"InvertedSouls,InvertedSouls","EarlyAccess,Action,Indie,Simulation,EarlyAcces...","Multi-player,OnlineMulti-Player,Co-op,OnlineCo...","Action,Indie,Simulation,EarlyAccess",rip,"Action,Indie,Simulation,EarlyAccessInvertedSou...","Action,Indie,Simulation,EarlyAccessEarlyAccess...","Action,Indie,Simulation,EarlyAccessEarlyAccess...","Action,Indie,Simulation,EarlyAccessInvertedSou..."
2915,The Deer,DodoDigital,"DodoDigital,DodoDigital","Gore,Adventure,Violent,Indie,Horror,Hunting,Fi...","Single-player,Steamislearningaboutthisgame \n\...","Adventure,Indie",thedeer,"Adventure,IndieDodoDigital,DodoDigitalDodoDigital","Adventure,IndieGore,Adventure,Violent,Indie,Ho...","Adventure,IndieGore,Adventure,Violent,Indie,Ho...","Adventure,IndieDodoDigital,DodoDigitalDodoDigi..."
2916,Tank Operations: European Campaign,LinkedDimensions,"rokapublish,rokapublish","EarlyAccess,Strategy,WorldWarII,Military,War,H...","Single-player,SteamAchievements,Steamislearnin...","Strategy,EarlyAccess",tankoperationseuropeancampaign,"Strategy,EarlyAccessrokapublish,rokapublishLin...","Strategy,EarlyAccessEarlyAccess,Strategy,World...","Strategy,EarlyAccessEarlyAccess,Strategy,World...","Strategy,EarlyAccessrokapublish,rokapublishLin..."
2917,King-Dom,maddma,"maddma,maddma","Strategy,Chess,Indie","OnlineMulti-Player,SteamAchievements,SteamWork...",Strategy,kingdom,"Strategymaddma,maddmamaddma","StrategyStrategy,Chess,Indiemaddma","StrategyStrategy,Chess,IndieOnlineMulti-Player...","Strategymaddma,maddmamaddmaOnlineMulti-Player,..."


In [49]:
final_df.head()

,name,developer,publisher,popular_tags,game_details,genre,ID,genre_publisher_developer,genre_popular_tags_developer,genre_popular_tags_game_details,genre_publisher_developer_game_details
0,DayZ,BohemiaInteractive,"BohemiaInteractive,BohemiaInteractive","Survival,Zombies,OpenWorld,Multiplayer,PvP,Mas...","Multi-player,OnlineMulti-Player,SteamWorkshop,...","Action,Adventure,MassivelyMultiplayer",dayz,"Action,Adventure,MassivelyMultiplayerBohemiaIn...","Action,Adventure,MassivelyMultiplayerSurvival,...","Action,Adventure,MassivelyMultiplayerSurvival,...","Action,Adventure,MassivelyMultiplayerBohemiaIn..."
1,EVE Online,CCP,"CCP,CCP","Space,MassivelyMultiplayer,Sci-fi,Sandbox,MMOR...","Multi-player,OnlineMulti-Player,MMO,Co-op,Onli...","Action,FreetoPlay,MassivelyMultiplayer,RPG,Str...",eveonline,"Action,FreetoPlay,MassivelyMultiplayer,RPG,Str...","Action,FreetoPlay,MassivelyMultiplayer,RPG,Str...","Action,FreetoPlay,MassivelyMultiplayer,RPG,Str...","Action,FreetoPlay,MassivelyMultiplayer,RPG,Str..."
2,TERA,"Bluehole,Inc.","EnMasseEntertainment,EnMasseEntertainment","FreetoPlay,MMORPG,MassivelyMultiplayer,RPG,Ope...","Multi-player,MMO,Co-op,SteamTradingCards,Parti...","Action,Adventure,FreetoPlay,MassivelyMultiplay...",tera,"Action,Adventure,FreetoPlay,MassivelyMultiplay...","Action,Adventure,FreetoPlay,MassivelyMultiplay...","Action,Adventure,FreetoPlay,MassivelyMultiplay...","Action,Adventure,FreetoPlay,MassivelyMultiplay..."
3,Stonehearth,RadiantEntertainment,"(none),(none)","CityBuilder,Building,Sandbox,Strategy,Survival...","Single-player,Multi-player,OnlineMulti-Player,...","Indie,Simulation,Strategy",stonehearth,"Indie,Simulation,Strategy(none),(none)RadiantE...","Indie,Simulation,StrategyCityBuilder,Building,...","Indie,Simulation,StrategyCityBuilder,Building,...","Indie,Simulation,Strategy(none),(none)RadiantE..."
4,Call of Duty®: Black Ops,Treyarch,"Activision,Activision","Action,FPS,Zombies,Multiplayer,Shooter,Singlep...","Single-player,Multi-player,Co-op,SteamAchievem...",Action,callofdutyblackops,"ActionActivision,ActivisionTreyarch","ActionAction,FPS,Zombies,Multiplayer,Shooter,S...","ActionAction,FPS,Zombies,Multiplayer,Shooter,S...","ActionActivision,ActivisionTreyarchSingle-play..."


In [50]:
tfv = TfidfVectorizer(min_df=3,  max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3),
            stop_words = 'english')


In [51]:
tfv_matrix = tfv.fit_transform(final_df['popular_tags'])
tfv_matrix

<2919x4280 sparse matrix of type '<class 'numpy.float64'>'
	with 66214 stored elements in Compressed Sparse Row format>

In [52]:
tfv_matrix.shape

(2919, 4280)

In [53]:
from sklearn.metrics.pairwise import sigmoid_kernel
sig = sigmoid_kernel(tfv_matrix, tfv_matrix)

In [54]:
sig[0]

array([0.76169226, 0.76160173, 0.76160009, ..., 0.76159942, 0.76159485,
       0.76159558])

In [55]:
indices = pd.Series(final_df.index, index=final_df['name']).drop_duplicates()
indices

name
DayZ                                     0
EVE Online                               1
TERA                                     2
Stonehearth                              3
Call of Duty®: Black Ops                 4
                                      ... 
RIP                                   3030
The Deer                              3032
Tank Operations: European Campaign    3033
King-Dom                              3034
GRID                                  3035
Length: 2919, dtype: int64

In [56]:
indices['DayZ']

0

In [57]:
list(enumerate(sig[indices['DayZ']]))


[(0, 0.7616922633426716),
 (1, 0.7616017259861846),
 (2, 0.7616000868010051),
 (3, 0.7616003921133147),
 (4, 0.7616043818188682),
 (5, 0.7615993304748822),
 (6, 0.7615962058516692),
 (7, 0.7616021353958156),
 (8, 0.7615972262762181),
 (9, 0.7615966042107922),
 (10, 0.7616032817923911),
 (11, 0.7616064853204866),
 (12, 0.761596709159686),
 (13, 0.7616048173055486),
 (14, 0.7616014117090106),
 (15, 0.7615958304136915),
 (16, 0.761598791110981),
 (17, 0.7615970805607031),
 (18, 0.7615975024917704),
 (19, 0.7615974488436473),
 (20, 0.7616015506341951),
 (21, 0.7615991576445733),
 (22, 0.7615975489233593),
 (23, 0.7615986851072869),
 (24, 0.7616003513720242),
 (25, 0.7615980585819383),
 (26, 0.7616017309691101),
 (27, 0.7616003558194403),
 (28, 0.7615970630336956),
 (29, 0.7615953906567686),
 (30, 0.761603719184123),
 (31, 0.7615970425593437),
 (32, 0.761597084659569),
 (33, 0.7616101389020133),
 (34, 0.7616016943926555),
 (35, 0.7615986361639068),
 (36, 0.7615979887479413),
 (37, 0.7615963

In [58]:
sorted(list(enumerate(sig[indices['DayZ']])), key=lambda x: x[1], reverse=True)


[(0, 0.7616922633426716),
 (2777, 0.761627316149172),
 (214, 0.7616267436409827),
 (125, 0.7616259537652906),
 (80, 0.7616223740631244),
 (2496, 0.7616222039007013),
 (482, 0.761616385161345),
 (740, 0.7616159153069912),
 (566, 0.7616155185335229),
 (638, 0.7616154774513444),
 (187, 0.7616145278407845),
 (2337, 0.7616132803188497),
 (112, 0.7616130877655052),
 (2640, 0.7616130119570564),
 (1003, 0.7616120783872664),
 (2794, 0.7616119319458989),
 (122, 0.7616115520762932),
 (2387, 0.761611406804574),
 (2624, 0.76161120440085),
 (373, 0.7616110299180198),
 (2472, 0.761610948687691),
 (1377, 0.761610868686472),
 (670, 0.7616107703591118),
 (494, 0.7616106465942734),
 (1349, 0.7616101821359009),
 (33, 0.7616101389020133),
 (2301, 0.7616101257744359),
 (1214, 0.7616099863772332),
 (2723, 0.7616098416832553),
 (229, 0.7616096402518305),
 (397, 0.7616096357019363),
 (468, 0.7616094269140102),
 (1403, 0.7616092589561176),
 (2604, 0.7616091008456323),
 (867, 0.7616090584525412),
 (2329, 0.76160

In [59]:
def give_rec(title, sig=sig):
    idx = indices[title]
    sig_scores = list(enumerate(sig[idx]))
    sig_scores = sorted(sig_scores, key=lambda x: x[1], reverse=True) 
    sig_scores = sig_scores[1:11]
    game_indices = [i[0] for i in sig_scores]
    return final_df['name'].iloc[game_indices]

In [60]:
give_rec('Call of Duty®: Black Ops')

41        Call of Duty®: Modern Warfare® 3
62        Call of Duty®: Modern Warfare® 2
622                             F.E.A.R. 3
2399             NecroVisioN: Lost Company
310                   Aliens vs. Predator™
2812             Frontlines™: Fuel of War™
2794                   Wanderlust: Rebirth
1774    Gotham City Impostors Free to Play
1556              The Haunted: Hells Reach
121       Counter-Strike: Global Offensive
Name: name, dtype: object